In [4]:
import os
import pandas as pd
from fpdf import FPDF
from datetime import datetime

team_folder_paths = {
    'CHI_DOG': '/Users/ajfoeckler/Downloads/chicago_games',
    'MIL_MIL1': '/Users/ajfoeckler/Downloads/milwaukee_games',
    'KAN_COU': '/Users/ajfoeckler/Downloads/kane_county_games',
    'SIO_CIT1': '/Users/ajfoeckler/Downloads/sioux_city_games',
    'FAR_RED': '/Users/ajfoeckler/Downloads/fargo_games',
    'SIO_FAL': '/Users/ajfoeckler/Downloads/SF_games',
    'KAN_CIT3': '/Users/ajfoeckler/Downloads/KC_games',
    'LIN_SAL': '/Users/ajfoeckler/Downloads/lincoln_games',
    'WIN_GOL':'/Users/ajfoeckler/Downloads/winnipeg_games',
    'GAR_SOU':'/Users/ajfoeckler/Downloads/gary_games',
    'SIO_CIT1':'/Users/ajfoeckler/Downloads/sioux_city_games',
    'CLE_RAI':'/Users/ajfoeckler/Downloads/cleburne_games',
    
    # add other teams here as needed…
}
selected_team = 'CHI_DOG'
folder_path = team_folder_paths[selected_team]
save_folder_path = '/Users/ajfoeckler/Downloads/chicago_hitting'

# Create the save directory if it doesn't exist
os.makedirs(save_folder_path, exist_ok=True)

team_pitchers = pd.read_csv('team_pitchers_tagged.csv')

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# === CONFIGURATION ===
pitcher_side = 'Left'    # or 'Left'
pitcher_abbrev = 'RHP' if pitcher_side == 'Right' else 'LHP'

# Load all CSV files and concatenate them into a single DataFrame
data_frames = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    data_frame = pd.read_csv(file_path)
    data_frame = data_frame[data_frame['PitcherThrows'] == pitcher_side] 
    data_frame['Date'] = pd.to_datetime(data_frame['Date'], errors='coerce')
    data_frame = data_frame.dropna(subset=['Date'])
    data_frame = data_frame[data_frame['Date'].dt.year == 2025]  # ✅ filter for 2025 only
    data_frames.append(data_frame)
    data_frames.append(data_frame)

original_data = pd.concat(data_frames, ignore_index=True)

# Print the columns of the original data for debugging
print("Columns in original_data:", original_data.columns)

# Ensure 'Date' column is in datetime format
original_data['Date'] = pd.to_datetime(original_data['Date'], errors='coerce')

# Normalize batter names
original_data['Batter'] = original_data['Batter'].replace({
    'Pruitt Jr, Reggie': 'Pruitt Jr, Reggie',

}).str.strip()

# Filter out rows with NaT in the 'Date' column
original_data = original_data.dropna(subset=['Date'])

# Check if 'BatterTeam' is in columns, if not print available columns
if 'BatterTeam' not in original_data.columns:
    print("Available columns in original_data:", original_data.columns)
    raise KeyError("Column 'BatterTeam' does not exist in the data.")

# Normalize batter names
original_data['Batter'] = original_data['Batter'].replace({
    'Pruitt, Reggie': 'Pruitt Jr, Reggie',

})

# Strip any leading or trailing whitespace from 'Batter' column
original_data['Batter'] = original_data['Batter'].str.strip()

# Create a dictionary mapping each batter to their team
batter_team_mapping = original_data[['Batter', 'BatterTeam']].drop_duplicates().set_index('Batter').to_dict()['BatterTeam']

# ——— manual override for batters who’ve switched teams ———
manual_batter_overrides = {

    # add more overrides here
}

# manual override: these hitters used to be on another club
manual_batter_original_team = {

}

for batter, new_team in manual_batter_overrides.items():
    batter_team_mapping[batter] = new_team
# ——————————————————————————————————————————————

filtered_data = original_data[(original_data['Balls'] == 0) & 
                              (original_data['Strikes'] == 0)].copy()

# Get a list of unique teams
unique_teams = filtered_data['BatterTeam'].unique()

# Prompt user to select a team
print("Available teams:", unique_teams)
selected_team = input("Please enter the team code to generate the report for: ").strip()

# figure out which folders to pull
teams_to_load = { selected_team } | set(manual_batter_original_team.values())
folders_to_load = [ team_folder_paths[t] for t in teams_to_load ]

# now read _all_ of those folders’ CSVs
data_frames = []
for folder in folders_to_load:
    for fn in os.listdir(folder):
        if not fn.endswith('.csv'): 
            continue
        df = pd.read_csv(os.path.join(folder, fn))
        df = df[df['PitcherThrows'] == pitcher_side]
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df[df['Date'].dt.year == 2025]  # ✅ Filter for 2025 here
        data_frames.append(df)

original_data = pd.concat(data_frames, ignore_index=True)

# 4) Now apply the first-pitch filter
filtered_data = original_data[
    (original_data['Balls'] == 0) &
    (original_data['Strikes'] == 0)
].copy()

# 5) Build mapping & override
batter_team_mapping = (
    filtered_data[['Batter','BatterTeam']]
    .drop_duplicates()
    .set_index('Batter')['BatterTeam']
    .to_dict()
)
for b, old in manual_batter_original_team.items():
    batter_team_mapping[b] = selected_team
    
# Define the strike zone dimensions
strike_zone_top = 3.673333
strike_zone_bottom = 1.524166667
strike_zone_left = -0.83083333
strike_zone_right = 0.83083333

# Function to process the data and calculate required metrics
def process_data(data):
    # Combine changeups and splitters
    data['AutoPitchType'] = data['AutoPitchType'].replace({'ChangeUp': 'Changeup/Splitter', 'Splitter': 'Changeup/Splitter'})

    # Filter out undefined TaggedHitType
    data = data[data['TaggedHitType'] != 'undefined']
    
    # Determine if a swing occurs
    data['Swing'] = data['PitchCall'].apply(lambda x: 1 if x in ['StrikeSwinging', 'FoulBallNotFieldable', 'InPlay'] else 0)
    
    # Determine if a whiff occurs (i.e., a swing and miss)
    data['Whiff'] = data['PitchCall'].apply(lambda x: 1 if x == 'StrikeSwinging' else 0)
    
    # Classify batted balls in play
    data['FlyBall'] = data['TaggedHitType'].apply(lambda x: 1 if x == 'FlyBall' else 0)
    data['GroundBall'] = data['TaggedHitType'].apply(lambda x: 1 if x == 'GroundBall' else 0)
    data['LineDrive'] = data['TaggedHitType'].apply(lambda x: 1 if x == 'LineDrive' else 0)
    data['Popup'] = data['TaggedHitType'].apply(lambda x: 1 if x == 'Popup' else 0)
    
    # Calculate barrels based on given criteria
    data['Barrel'] = ((data['ExitSpeed'] >= 95) & 
                      (data['Angle'] >= 5) & 
                      (data['Angle'] <= 37)).astype(int)
    
    # Determine if a pitch is inside or outside the strike zone
    data['InStrikeZone'] = ((data['PlateLocHeight'] <= strike_zone_top) &
                            (data['PlateLocHeight'] >= strike_zone_bottom) &
                            (data['PlateLocSide'] >= strike_zone_left) &
                            (data['PlateLocSide'] <= strike_zone_right)).astype(int)
    
    # Calculate chases (i.e., swings at pitches outside the strike zone)
    data['Chase'] = (data['Swing'] & (data['InStrikeZone'] == 0)).astype(int)
    
    # Calculate batted balls in play
    data['BattedBallInPlay'] = data[['FlyBall', 'GroundBall', 'Popup', 'LineDrive']].sum(axis=1)
    
    # Group by Batter and AutoPitchType for calculation
    grouping_columns = ['Batter', 'AutoPitchType']
    
    # Calculate metrics grouped by Batter and AutoPitchType
    metrics = data.groupby(grouping_columns).agg({
        'FlyBall': 'sum',
        'GroundBall': 'sum',
        'Popup': 'sum',
        'LineDrive': 'sum',
        'Barrel': 'sum',
        'Swing': 'sum',
        'Whiff': 'sum',
        'Chase': 'sum',
        'InStrikeZone': 'sum',
        'BattedBallInPlay': 'sum',
        'PitchNo': 'count'  # Count the total number of pitches
    }).reset_index()
    
    # Calculate additional swing metrics
    z_swing = data[data['InStrikeZone'] == 1].groupby(grouping_columns)['Swing'].sum().reset_index(name='Z_Swing')
    z_total = data[data['InStrikeZone'] == 1].groupby(grouping_columns)['PitchNo'].count().reset_index(name='Z_Total')
    o_swing = data[data['InStrikeZone'] == 0].groupby(grouping_columns)['Swing'].sum().reset_index(name='O_Swing')
    o_total = data[data['InStrikeZone'] == 0].groupby(grouping_columns)['PitchNo'].count().reset_index(name='O_Total')
    
    metrics = metrics.merge(z_swing, on=grouping_columns, how='left')
    metrics = metrics.merge(z_total, on=grouping_columns, how='left')
    metrics = metrics.merge(o_swing, on=grouping_columns, how='left')
    metrics = metrics.merge(o_total, on=grouping_columns, how='left')
    
    # Calculate percentages and round to the nearest hundredth
    metrics['fly_ball_pct'] = (metrics['FlyBall'] / metrics['BattedBallInPlay'] * 100).round(2)
    metrics['ground_ball_pct'] = (metrics['GroundBall'] / metrics['BattedBallInPlay'] * 100).round(2)
    metrics['line_drive_pct'] = (metrics['LineDrive'] / metrics['BattedBallInPlay'] * 100).round(2)
    metrics['barrel_pct'] = (metrics['Barrel'] / metrics['BattedBallInPlay'] * 100).round(2)
    metrics['swing_pct'] = (metrics['Swing'] / metrics['PitchNo'] * 100).round(2)
    metrics['whiff_pct'] = (metrics['Whiff'] / metrics['Swing'] * 100).round(2)
    metrics['z_swing_pct'] = (metrics['Z_Swing'] / metrics['Z_Total'] * 100).round(2)
    metrics['o_swing_pct'] = (metrics['O_Swing'] / metrics['O_Total'] * 100).round(2)
    metrics['chase_pct'] = (metrics['Chase'] / metrics['PitchNo'] * 100).round(2)  # Add this line
    
    # Add total batted balls column
    metrics['total_batted_balls'] = metrics['FlyBall'] + metrics['GroundBall'] + metrics['LineDrive'] + metrics['Popup']
    
    # Retain the required columns
    percentage_columns = [
        'Batter', 'AutoPitchType',
        'PitchNo',
        'total_batted_balls',  # Add this column to the final output
        'swing_pct',
        'chase_pct'
    ]
    metrics = metrics[percentage_columns]
    
    return metrics

# Function to create a PDF report
class PDF(FPDF):
    def __init__(self, team, vs_hand, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.team = team
        self.vs_hand = vs_hand

    def header(self):
        self.set_font('Arial', 'B', 12)
        # now shows, e.g., "Team: KAN_COU vs. RHP"
        self.cell(0, 10, f'Team: {self.team} vs. {self.vs_hand}', 0, 1, 'C')
    
    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(10)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def table(self, dataframe, title):
        self.set_font('Arial', 'B', 8)  # Reduced font size
        self.cell(0, 10, title, 0, 1, 'C')
        self.ln(2)  # Reduced space between title and table
        
        # Calculate the optimal width for each column
        page_width = self.w - 2 * self.l_margin
        col_widths = [page_width / len(dataframe.columns)] * len(dataframe.columns)
        row_height = self.font_size * 1.2  # Reduced row height
        
        # Add table headers
        for col_name, col_width in zip(dataframe.columns, col_widths):
            self.cell(col_width, row_height, str(col_name), border=1)
        self.ln(row_height)
        
        self.set_font('Arial', '', 8)  # Reduced font size for data
        for row in dataframe.itertuples(index=False):
            for item, col_width in zip(row, col_widths):
                self.multi_cell(col_width, row_height, str(item), border=1, ln=3, max_line_height=self.font_size)
            self.ln(row_height)

def generate_pdf_report(dataframe, team, output_path, batter_team_mapping, vs_hand):
    pdf = PDF(team, vs_hand)
    # Filter data by team using batter_team_mapping
    batters_in_team = [batter for batter, batter_team in batter_team_mapping.items() if batter_team == team]
    team_data = dataframe[dataframe['Batter'].isin(batters_in_team)]

    # Exclude specific batters
    excluded_batters = ["Cespedes, Yoelqui", "Bell, Brantley", "Soto, Jonathan", "Goosenberg, Shawn", "Martin, Robby", "Scolan, Matthew", "Moris, Max", "Adolfo, Micker", "Amaya, Carlos", "Chalus Jr., Eric", "Diaz, Juan", "Marte, Jefry", "Mezquita, Jhordany", "Ostberg, Erik", "Santiago, Glenn", "Penzetta, Nick", "Voss, Joe", "Bonifacio, Jorge", "Jackson, Darryl", "Parker, Cade", "Broadway, Taylor", "Allgeyer, Jacob", "Machado, Carlos", "Noriega, Andres", "Henley, Blair", "Howell, Korry", "McDowell, Theo", "Scott, Kristian", "Contreras, Jose", "Schmack, Kyle", "Williams, Donivan", "Ultsch, Nick", "Cribbs, Galli", "Dunhurst, Hayden", "Phillips, Brett", "Rijo, Nilo", "Upshaw, Caleb", "Valdez, C.J.", "Jones, Thomas", "McDaniel, Chase", "Novak, Nick", "Pettigrew, Zion", "Ward, Je'Von", "Sparks, Lamar", "Palmeiro, Preston", "Davis, Jaylin", "Law, Trey", "Aldrete, Carter", "Alexander, Hill", "Berglund, Michael", "Bigford, Trey", "Boswell, Bret", "Brocato, Anthony", "Brothers, Blaze", "Cantleberry, Jake", "Dillard, Thomas", "Duzenack, Camden", "Groshans, Jaxx", "Holland, Korey", "Mechals, Kade", "O'Grady, Brian", "Palmeiro, Preston", "Velez, Antonio", "Washington, Mark", "Davis, Jaylin", "Givin, Matt", "Huckstorf, Kyle", "Layer, Abdiel", "Henson, Spencer", "Dragum, Jack", "Denning, Connor", "Merkel, Tyler", "Green, Thomas", "Alexander, Evan", "Davis, Colten", "Dykhoff, Jake", "Green, Thomas", "Connell, Justin", "vooletich, zac", "Arroyo, Edwin", "Croes, Dayson", "Gonzalez, Marcos", "Emery, Robert", "Enriquez, Roby", "Galan, Yosy", "Marcos, Gonzales", "Hoover, Jake", "McCurdy, Carson", "Sierra, Miguelangel", "Stroh, Parker", "Waite, Jonathan", "McMurray, Jake", "Schwabe, Cadyn", "Simington, Miles", "Byrd, Justin", "Farmer, Justin", "Maiben, Derek", "Reeves, Dalton", "Barfield, Jacob", "Busch, Will", "Corona, Emilio", "Gomez, Dario", "Green, Jake", "Jackson, Jaren", "Lingua, Daniel", "Martinez, Osvaldo", "Meyer, Jake", "Montano, Daniel", "Nogowski, John", "Perez, Daniel",  "Womack, Alsander", "Dorighi, Brennen", "Childs, Dwight", "Drury, Austin", "Perez, Henderson", "Ramon, Amos", "Wetherbee, Jared", "Zinn, Delvin", "Bockelie, Jacob", "Decker, Will", "Fields, Brandon", "Gulino, Michael", "Hurd, Aaron", "Perez, Yanio", "Perez, Mikey", "Smith, Chad", "Calarco, Anthony", "Campagna, Joe", "Grant-Parks, Blake", "Maberry, David", "Mcarthur, General", "Parks, Pavin", "Aguilar, Bryan", "Amaral, Danny", "Armaral, Danny", "Milam, Kevin", "Ortiz, Channey", "Hjelle, Jake", "Phillips, Dakota", "Siket, Jordan", "Christopher, Shamoy", "Eickhoff, Logan", "Foster, Kendall", "Meiners, Tate", "Seay, Mason", "Brusa, Gio", "Hall, Adam", "Holgate, Ryan", "Turner, Braxton", "Escala, Willie", "Rodriguez, Brett", "Smith, Armani", "Cushing, Jared", "Del Valle, Francisco", "Panzetta, Nick", "Barranca, Antonio", "Pike, Chad", "Marrero, Wendell", "Costes, Marty", "Crook, Narciso", "Harris, Chase", "Abbatine, Anthony", "Diaz, Gio", "Levari, Kenneth", "Quintana, Guillermo", "Rincon, Carlos", "Valera, Jackson", "Law, Trey", "Sermo, Jose", "Taylor, John", "Cannon, Cameron", "Davis, Jonah", "Randolph, Cornelius", "Reid, Simon", "Smith, Harrison", "Reyes, Bryan", "Doersching, Griffin", "Coulter, Clint", "Sosa, Gus", "Nicholas, Novak", "Morales, Roy", "Alcantara, Ismael", "Dexter, Sam", "Valdez, CJ", "Livorsi, Ben", "Meza, Eric", "Quiggle, Kona", "Ward, Drew", "Mount, Drew", "Sarringar, Spencer", "Spence, Liam", "Ulrich, Wyatt", "Cedrola, Lorenzo", "Gomez, Moises", "O'Conner, Justin", "Avelino, Abiatal", "Cannon, Cam", "Maxwell, Carson", "Amaral, Daniel", "Bradley, Tucker", "Fry, Jared", "Anderson, Nick", "Awtry, Marshall", "Awtry , Marshall", "Baeza, Alex", "Conners, Dakota", "DeVine, Drew", "Epp, Alex", "Hewitt, Max", "Mattis, Gary", "Ortega, Jake", "Pita, Matt", "Roskam, Luke", "Takacs, Aaron", "Williams, Logan", "Zurbrugg, Zane"]
    team_data = pd.concat([team_data, dataframe[dataframe['Batter'].isin(["Clark, Tripp", "Williams, Jaylyn"])]])
    team_data = team_data[~team_data['Batter'].isin(excluded_batters)]

    # Rename columns for the table in the PDF
    rename_columns = {
        'swing_pct': 'Swing%',
        'chase_pct': 'Chase%',
        'PitchNo': '# of Pitches',
        'total_batted_balls': 'Total Batted Balls'
    }
    
    team_data = team_data.rename(columns=rename_columns)

    # Include 'BatterSide' in the team_data DataFrame
    team_data = team_data.merge(original_data[['Batter', 'BatterSide']].drop_duplicates(), on='Batter', how='left')

    # Ensure no duplicate rows for the same batter and pitch type
    team_data = team_data.drop_duplicates(subset=['Batter', 'AutoPitchType'])

    # Define the pitch order
    pitch_order = ['Four-Seam', 'Sinker', 'Curveball', 'Slider', 'Changeup/Splitter', 'Cutter']

    batters = team_data['Batter'].unique()
    for batter in batters:
        batter_data = team_data[team_data['Batter'] == batter]
        batter_side = batter_data['BatterSide'].iloc[0]  # Get the batter side for the batter
        pdf.add_page()
        pdf.chapter_title(f'Batter: {batter} (Side: {batter_side})')
        
        # Sort by the defined pitch order
        pitch_data_sorted = batter_data.set_index('AutoPitchType').reindex(pitch_order).reset_index()

        for pitch_type in pitch_order:
            pitch_data = pitch_data_sorted[pitch_data_sorted['AutoPitchType'] == pitch_type]
            if not pitch_data.empty:
                # Swing Tendencies Table
                swing_tendencies = pitch_data[['# of Pitches', 'Swing%', 'Chase%', 'Total Batted Balls']]
                pdf.table(swing_tendencies, title=f'1st Pitch Swing Tendencies for {pitch_type}')
                
                pdf.ln(1.5)  # Add reduced space between tables
    
    pdf.output(output_path)

metrics_df = process_data(filtered_data)
output_pdf = os.path.join(save_folder_path,
                          f'{selected_team}_first_pitch_{pitcher_abbrev}.pdf')
generate_pdf_report(metrics_df,
                    selected_team,
                    output_pdf,
                    batter_team_mapping,
                    pitcher_abbrev)

print(f"PDF report created: {output_pdf}")


Columns in original_data: Index(['PitchNo', 'Date', 'Time', 'PAofInning', 'PitchofPA', 'Pitcher',
       'PitcherId', 'PitcherThrows', 'PitcherTeam', 'Batter',
       ...
       'ThrowTrajectoryZc1', 'ThrowTrajectoryZc2', 'PitchReleaseConfidence',
       'PitchLocationConfidence', 'PitchMovementConfidence',
       'HitLaunchConfidence', 'HitLandingConfidence',
       'CatcherThrowCatchConfidence', 'CatcherThrowReleaseConfidence',
       'CatcherThrowLocationConfidence'],
      dtype='object', length=167)
Available teams: ['CHI_DOG' 'FAR_RED' 'KAN_COU' 'LAK_COU10' 'GAR_SOU' 'SIO_FAL' 'CLE_RAI'
 'KAN_CIT3' 'LIN_SAL' 'MIL_MIL1' 'WIN_GOL' 'SIO_CIT1']
Please enter the team code to generate the report for: CHI_DOG
PDF report created: /Users/ajfoeckler/Downloads/chicago_hitting/CHI_DOG_first_pitch_LHP.pdf


/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30678/2478121255.py:250: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30678/2478121255.py:252: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, f'Team: {self.team} vs. {self.vs_hand}', 0, 1, 'C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30678/2478121255.py:255: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_30678/2478121255.py:256: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, titl